In [14]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg


def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
dataset = 'furniture_train'
boxFilePath = '/media/data/chnxi/GOD/threshold_bboxes/'
mapFilePath = '/media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/'
outAnnoFolder = '/media/data/chnxi/GOD/json_annotatinos/'
orgAnnoFile = '/media/data/chnxi/HomeFurniture/json_annotations/train_annotations.json'
outAnnoFile = osp.join(outAnnoFolder, 'tmp_{}_annotations.json'.format(dataset))

visPath = '/media/data/chnxi/GOD/visualizations/'
visFolder = osp.join(visPath, dataset)
checkMkdir(visFolder)

modelSetMap = {'Fashion_Detector'  : 'fashion',
               'HF_Detector'       : 'furniture',
               'COCO_Mask_Detector': 'COCO',
               'OpenImage_Detector': 'Open Image'}

if dataset == 'furniture_train':
    gtSetName = 'furniture'
    mapFile = mapFilePath + 'HF_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'furniture_train.json',
                'COCO_Mask_Detector': 'furniture_train.pkl',
                'OpenImage_Detector': 'furniture_train.pkl'}
elif dataset == 'FashionV2_train':
    gtSetName = 'fashion'
    mapFile = mapFilePath + 'Fashion_dataset_sources_ids.pkl'
    srcFiles = {'HF_Detector'       : 'FashionV2_train.pkl',
                'COCO_Mask_Detector': 'FashionV2_train.pkl',
                'OpenImage_Detector': 'FashionV2_train.pkl'}
elif dataset == 'OpenImage_train':
    gtSetName = 'Open Image'
    mapFile = mapFilePath + 'OpenImage_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'OpenImage_train.json',
                'HF_Detector'       : 'OpenImage_train.pkl',
                'COCO_Mask_Detector': 'OpenImage_train.pkl'}
elif dataset == 'COCO_trainval':
    gtSetName = 'COCO'
    mapFile = mapFilePath + 'COCO_dataset_sources_ids.pkl'
    srcFiles = {'Fashion_Detector'  : 'COCO_trainval.json',
                'HF_Detector'       : 'coco_2014_trainval.pkl',
                'OpenImage_Detector': 'COCO_trainval.pkl'}

# for modelName in srcFiles:
#     box_file_name = srcFiles[modelName]
#     boxFileName = osp.join(boxFilePath, modelName, box_file_name)
#     print ("boxFileName = {}".format(boxFileName))
    
    
    

In [2]:
if dataset in ['furniture_train', 'FashionV2_train', 'OpenImage_train']:
    print ("creating json dataset {}".format(dataset))
    ds = JsonDataset(dataset)
    roidb = ds.get_roidb(gt=True)
elif dataset == 'COCO_trainval':
    datasetNames = ('coco_2014_train','coco_2014_valminusminival')
    roidb = []
    for dsName in datasetNames:
        ds = JsonDataset(dsName)
        rdb = ds.get_roidb(gt=True)
        roidb = roidb + rdb
imgNum = len(roidb)
print ('imgNum = {}'.format(imgNum))

creating json dataset furniture_train
================= loading annotation furniture_train
loading annotations into memory...
Done (t=7.03s)
creating index...
index created!
getting roidb
Done (t=97.45s)
imgNum = 127068


In [3]:
print (roidb[0]['image'])
print (roidb[-1]['image'])

/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/HomeFurniture/Images/17B0E85E02491E313EF2D908471730BD98FC5D553368E3F7A9F2AB3C25FF05FDB6C24FB7B7A1A7066AE04A2909ADB593E1925D7A6079E81444A3B3E9F1E4A8E3.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/HomeFurniture/Images/CE29382FB5DA946DC2BB68A1AF5A01A1FF28D472D3432A0F6FF0807DAD887B38BEC061FED7A3C4892FDB514AE66C0638E50C3284DCE37CB6FC788CC906A4CD8F.jpg


In [15]:
print ("loading {}".format(mapFile))
src_god_map = pkl.load(open(mapFile,'rb'))
print (src_god_map.keys())
cocoMap = src_god_map['name_to_god_id']['COCO']
print (cocoMap)
cocoIdMap = src_god_map['id_map']['COCO']
print(cocoIdMap)
print (len(cocoMap))

loading /media/data/chnxi/GOD/mappings/src_nameid_to_GOD_id_mappings/HF_dataset_sources_ids.pkl
dict_keys(['id_map', 'name_to_god_id'])
{'teddy bear': 543, 'bicycle': 548, 'apple': 281, 'laptop': 112, 'tennis racket': 516, 'donut': 263, 'snowboard': 511, 'carrot': 287, 'zebra': 82, 'keyboard': 109, 'scissors': 480, 'fire hydrant': 245, 'mouse': 111, 'airplane': 544, 'boat': 566, 'kite': 501, 'sheep': 77, 'horse': 58, 'cup': 339, 'traffic light': 572, 'toaster': 140, 'bowl': 334, 'book': 466, 'elephant': 51, 'orange': 296, 'stop sign': 570, 'knife': 343, 'pizza': 275, 'fork': 341, 'hair drier': 129, 'sandwich': 278, 'umbrella': 237, 'banana': 284, 'bus': 549, 'toothbrush': 409, 'spoon': 354, 'giraffe': 53, 'cell phone': 148, 'broccoli': 285, 'frisbee': 494, 'remote': 153, 'surfboard': 514, 'cow': 47, 'hot dog': 268, 'car': 550, 'skateboard': 509, 'dog': 49, 'cat': 45, 'baseball glove': 490, 'skis': 510, 'train': 561, 'truck': 555, 'parking meter': 455, 'suitcase': 161, 'cake': 251, 'win

In [5]:
print ("Loading model bboxes!")
srcBoxes = {}
for modelName in srcFiles:
    box_file_name = srcFiles[modelName]
    boxFileName = osp.join(boxFilePath, modelName, box_file_name)
    print ("Loading boxFileName = {}".format(boxFileName))
    if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
        # format: all_boxes[imgid][class_id]
        all_boxes = load_object(boxFileName)
        print ("loaded")
        print ("all_boxes.size = ({}, {})".format(len(all_boxes), len(all_boxes[0])))
        srcBoxes[modelName] = all_boxes
    elif modelName == 'OpenImage_Detector':
        # all_boxes[file_name] = {'file_name': file_name,
        #        'boxes': boxes,
        #        'label_names': label_names,
        #        'labels': labels}
        all_boxes = load_object(boxFileName)
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        srcBoxes[modelName] = all_boxes
    elif '.json' in box_file_name:
        # {'image_id': 'd7a0b91d2d83d457', 
        #'boxes': [{'score': 0.228, 'category_id': 2, 
        #'bbox': [506.49920000000003, 414.4128, 657.8248, 544.1536], 
        #'category_name': 'accessories-glasses'}], 'image_file_name': 'd7a0b91d2d83d457.jpg'}
        print ("loading {}".format(boxFileName))
        dets = json.load(open(boxFileName,'r'))
        all_boxes = {}
        for ix, det in enumerate(dets):
            # make it a dictionary with image file name as key
            img_file_name = det['image_file_name']
            all_boxes[img_file_name] = det['boxes']
        print ("len(all_boxes) = {}".format(len(all_boxes)))
        print ("loaded")
        srcBoxes[modelName] = all_boxes #dets
    print ("srcBoxes[{}] loaded".format(modelName))

Loading model bboxes!
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/Fashion_Detector/furniture_train.json
loading /media/data/chnxi/GOD/threshold_bboxes/Fashion_Detector/furniture_train.json
len(all_boxes) = 79277
loaded
srcBoxes[Fashion_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/COCO_Mask_Detector/furniture_train.pkl
loaded
all_boxes.size = (127068, 80)
srcBoxes[COCO_Mask_Detector] loaded
Loading boxFileName = /media/data/chnxi/GOD/threshold_bboxes/OpenImage_Detector/furniture_train.pkl
len(all_boxes) = 127068
srcBoxes[OpenImage_Detector] loaded


In [16]:
imgName = '2988531ACD30C78DC4C1F1EA859ED1C02DDF7773F9CBC67FFAD71F25A2CAD0D688A35E26D72C7525A0C4100ED0C9C3CA008942A59F6B9173D1E3552387B19AF3.jpg'
print ("srcBoxes['COCO_Mask_Detector'][10][14][1] = {}\n".format(srcBoxes['COCO_Mask_Detector'][10][14][1]))
print ("srcBoxes['Fashion_Detector'][imgName] = {}\n".format(srcBoxes['Fashion_Detector'][imgName]))
print ("srcBoxes['OpenImage_Detector'][imgName] = {}\n".format(srcBoxes['OpenImage_Detector'][imgName]))

srcBoxes['COCO_Mask_Detector'][10][14][1] = [279.27884   59.576763 296.89883   74.03485    0.592858]

srcBoxes['Fashion_Detector'][imgName] = [{'score': 0.5268, 'category_id': 48, 'bbox': [423.63, 761.9339, 475.65, 829.0], 'category_name': 'shoes-boots'}, {'score': 0.5247, 'category_id': 48, 'bbox': [593.01, 177.9034, 633.87, 235.7676], 'category_name': 'shoes-boots'}]

srcBoxes['OpenImage_Detector'][imgName] = {'file_name': '2988531ACD30C78DC4C1F1EA859ED1C02DDF7773F9CBC67FFAD71F25A2CAD0D688A35E26D72C7525A0C4100ED0C9C3CA008942A59F6B9173D1E3552387B19AF3.jpg', 'boxes': array([], dtype=float64), 'label_names': [], 'labels': array([], dtype=float64)}



In [17]:
#############   Creating taxonomy   #############################################
import pandas as pd
labelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_V1.tsv'
god_data = pd.read_csv(labelFile, delimiter='\t')
god_name_id_map = dict(zip(god_data['GOD_v1_name'], god_data['GOD_v1_id']))
god_classes = list(god_data['GOD_v1_name'])
print ("len(god_classes) original = {}".format(len(god_classes)))
god_classes = ['__background__'] + god_classes

len(god_classes) original = 581


In [18]:
data = {}
supercat = 'Generic'
data['info'] = {'description':'Bing Generic Object Detection dataset on {}'.format(dataset)}
data['type'] = 'instances'

data['categories'] = []

for cls, id in god_name_id_map.items():
    cats = {'supercategory':supercat,
            'id':id,
            'name':cls}
    data['categories'].append(cats)

print (data['categories'][:10])

[{'supercategory': 'Generic', 'id': 1, 'name': 'animal\\amphibian\\frog'}, {'supercategory': 'Generic', 'id': 2, 'name': 'animal\\arthropod\\arachnid\\scorpion'}, {'supercategory': 'Generic', 'id': 3, 'name': 'animal\\arthropod\\arachnid\\spider'}, {'supercategory': 'Generic', 'id': 4, 'name': 'animal\\arthropod\\crustacean\\crab'}, {'supercategory': 'Generic', 'id': 5, 'name': 'animal\\arthropod\\crustacean\\lobster'}, {'supercategory': 'Generic', 'id': 6, 'name': 'animal\\arthropod\\crustacean\\shrimp'}, {'supercategory': 'Generic', 'id': 7, 'name': 'animal\\arthropod\\insect\\ant'}, {'supercategory': 'Generic', 'id': 8, 'name': 'animal\\arthropod\\insect\\bee'}, {'supercategory': 'Generic', 'id': 9, 'name': 'animal\\arthropod\\insect\\butterfly'}, {'supercategory': 'Generic', 'id': 10, 'name': 'animal\\arthropod\\insect\\caterpillar'}]


In [10]:
imgNum = len(roidb)
# imgIds = range(imgNum)

In [19]:
import matplotlib.pyplot as plt
color_list = ['g', 'r', 'b', 'm']
def visImageClsThrshs(plt, splot, im, imgBoxes, ecolor='m', title='', showCateg=False):
    #plt.cla()
    fx, fy, fi = splot
    plt.subplot(fx, fy, fi)
    plt.imshow(im)
    plt.axis('off')
    plt.title(title)
    ax = plt.gca()
    for obj in imgBoxes:
        #print obj
        bbox = obj['bbox']
        categ_id = obj['category_id']
        categ = god_classes[categ_id].split('\\')[-1]
        #ecolor = 'm' #if gid == jmax else gtcolor
        #print ("{}: bbox = {}".format(categ, bbox))
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2],
                          bbox[3], fill=False,
                          edgecolor=ecolor, linestyle='-', linewidth=1))
        if showCateg:
            ax.text(bbox[0], bbox[1]-2, '{}'.format(categ), bbox={'facecolor': ecolor, 'alpha': 0.5})
    plt.draw()


def getAnno(image_id, bbox, categ_id, box_id):
    boxw = float(bbox[2]-bbox[0]) # convert np.ndarray.float32 to float in case json.dump not serializable
    boxh = float(bbox[3]-bbox[1])
    nbbox = [float(bbox[0]), float(bbox[1]), boxw, boxh]
    anno = {'segmentation':[],
            'area': boxw*boxh,
            'iscrowd':0,
            'image_id':image_id,
            'bbox':nbbox, # because COCO format [l, t, w,h] #float(r),float(b)],
            'category_id':categ_id,
            'id':box_id
            }
#     print ("anno.segmentation.type = {}".format(type(anno['segmentation'])))
#     print ("anno.bbox.type = {}".format(type(anno['bbox'])))
#     print ("anno.area.type = {}".format(type(anno['area'])))
#     print ("anno.iscrowd.type = {}".format(type(anno['iscrowd'])))
#     print ("anno.image_id.type = {}".format(type(anno['image_id'])))
#     print ("anno.category_id.type = {}".format(type(anno['category_id'])))
#     print ("anno.id.type = {}".format(type(anno['id'])))
    return anno

# from random import randint
# dsIdx = randint(0,imgNum-1)
# print(dsIdx)
# imgIds = [dsIdx]
# print (imgIds)

import random
visNum = 200
imgIds = random.sample(range(len(roidb)), visNum)
print(imgIds)

vis = True
saveVis = True
totalBoxCnt = 0
data["images"] = []
data["annotations"] = []

for imgid in imgIds: #imgid, entry in enumerate(roidb):
    visAnno = {} 
    entry = roidb[imgid]
    ###############################################
    ## data['images']
    ###############################################
    file_name = entry['image']
    imgName = osp.basename(file_name)
    imgH = entry['height']
    imgW = entry['width']
    img = {'file_name': imgName,
           'height': float(entry['height']),
           'width': float(entry['width']),
           'id':imgid
           }
    data["images"].append(img)
    print (img)
    print ("img.file_name.type = {}".format(type(img['file_name'])))
    print ("img.height.type = {}".format(type(img['height'])))
    print ("img.width.type = {}".format(type(img['width'])))
    print ("img.id.type = {}".format(type(img['id'])))
    ###############################################
    ## Add GT Boxes
    ###############################################
    src_god_id_map = src_god_map['id_map'][gtSetName]
    src_name_godid_map = src_god_map['name_to_god_id'][gtSetName]
    visName = gtSetName + '_GT'
    visAnno[visName] = []
    gt_boxes = entry['boxes']
    gt_src_cls_ids = entry['gt_classes']
    for bid, box in enumerate(gt_boxes):
        src_class_id = gt_src_cls_ids[bid]
        if src_class_id in src_god_id_map:
            god_class_id = src_god_id_map[src_class_id]
            print ("======{}:{} ==> GOD:{} {}".format(gtSetName, src_class_id, god_class_id, god_classes[god_class_id]))
            anno = getAnno(imgid, box[:4], god_class_id, totalBoxCnt)
            #print (anno)
            data["annotations"].append(anno)
            if vis:
                visAnno[visName].append(anno)
            totalBoxCnt += 1
    ###############################################
    ## Add boxes from other models
    ###############################################
    for modelName in srcFiles:
        modelSetName = modelSetMap[modelName]
        src_god_id_map = src_god_map['id_map'][modelSetName]
        src_name_godid_map = src_god_map['name_to_god_id'][modelSetName]
        visAnno[modelName] = []
#         print ("{} ==> {}".format(modelName, modelSetName))
        if modelName in ['COCO_Mask_Detector', 'HF_Detector']:
            # format: all_boxes[imgid][class_id]
            for cid, cls_boxes in enumerate(srcBoxes[modelName][imgid]):
                src_class_id = cid + 1 # all_boxes starts from 0
                if src_class_id in src_god_id_map and len(cls_boxes) > 0:
                    god_class_id = src_god_id_map[src_class_id]
                    print ("========{}:{} ==> GOD:{}:{}".format(modelSetName, src_class_id, god_class_id, god_classes[god_class_id]))
                    for bbox in cls_boxes:
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #if totalBoxCnt % 10 == 0:
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno)
                        totalBoxCnt += 1
        if modelName == 'OpenImage_Detector':
            # all_boxes[file_name] = {'file_name': file_name,
            #        'boxes': boxes,
            #        'label_names': label_names,
            #        'labels': labels}
            boxes = srcBoxes[modelName][imgName]['boxes']
            label_names = srcBoxes[modelName][imgName]['label_names']
            for bid, box in enumerate(boxes):
                src_class_name = label_names[bid] # do not use labels: it's not correct for OpenImage Id
                if src_class_name in src_name_godid_map:
                    god_class_id = src_name_godid_map[src_class_name]
                    print ("========= {}:{} ==> GOD:{}:{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                    bbox = [box[0] * imgW, box[1] * imgH, box[2] * imgW, box[3] * imgH]
                    anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                    #print (anno)
                    data["annotations"].append(anno)
                    if vis:
                        visAnno[modelName].append(anno) 
                    totalBoxCnt += 1                 
        if modelName == 'Fashion_Detector': #'.json' in box_file_name:
            # all_boxes[imgName] = [ {[506.49920000000003, 414.4128, 657.8248, 544.1536], 
            #'category_name': 'accessories-glasses'} ]
            if imgName in srcBoxes[modelName]:
                boxes = srcBoxes[modelName][imgName]
                for bid, obj in enumerate(boxes):
                    src_class_name = obj['category_name']
                    if src_class_name in src_name_godid_map:
                        print (obj)
                        god_class_id = src_name_godid_map[src_class_name]
                        print ("========{}:{} ==> GOD:{}{}".format(modelSetName, src_class_name, god_class_id, god_classes[god_class_id]))
                        bbox = obj['bbox']
                        anno = getAnno(imgid, bbox, god_class_id, totalBoxCnt)
                        #print (anno)
                        data["annotations"].append(anno)
                        if vis:
                            visAnno[modelName].append(anno) 
                        totalBoxCnt += 1    

    #################################
    # visualization of merging boxes
    #################################
    if vis:
        im = Image.open(file_name)
        f, axarr = plt.subplots(1, 4, sharey=True, figsize=(20, 20))
        plt.cla()
        fid = 0
        for modelName in visAnno: #['prec_at_0.9']: #new_cls_thrsh_dict:
            print (modelName)
            ax = axarr[fid]
            visImageClsThrshs(plt, (1,4, fid+1), im, visAnno[modelName], ecolor=color_list[fid], title=modelName, showCateg=True)
            fid += 1
        if saveVis: # and len(visAnno['COCO_Mask_Detector']) > 0:
            plt.savefig(osp.join(visFolder,imgName), bbox_inches='tight',pad_inches = 0)
        plt.show()
        plt.close()

[120433, 102660, 103520, 19202, 42268, 62458, 46995, 7922, 86974, 23409, 125334, 111350, 41191, 78065, 28414, 10535, 33888, 121398, 108045, 77685, 62650, 72627, 65966, 63953, 99902, 74470, 39580, 61808, 34653, 16093, 75864, 81604, 64563, 97830, 18419, 79589, 110448, 9202, 49519, 93176, 40193, 68870, 12524, 91759, 1542, 38812, 73716, 5637, 35635, 42930, 47342, 15103, 104543, 107782, 103238, 378, 113588, 5207, 82127, 41566, 25317, 63583, 29193, 79753, 98624, 71910, 11370, 9076, 23190, 20619, 49489, 79719, 120399, 27707, 102168, 58103, 100857, 110285, 24946, 3380, 120469, 3412, 100763, 91601, 73472, 2492, 42331, 125806, 112707, 123970, 92960, 114392, 110080, 120830, 65583, 69478, 112078, 67344, 85243, 105022, 104063, 43963, 53379, 7125, 43351, 114150, 19026, 54238, 49957, 40455, 122704, 109238, 106359, 13672, 14173, 37229, 60049, 32858, 95328, 88647, 48748, 52472, 30481, 99698, 27484, 74056, 77965, 61002, 25077, 108784, 112736, 25852, 119238, 75828, 33165, 113690, 106290, 85154, 50419, 85

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'EFA42CD7CC1BB3F25468B0F6D82A50147703AAF21E47ADDC041C85FB74C2D641900A821270DE9D7B1D2B8DE939A996F109F5A01CA1AB5659A58077B7DF3203B7.jpg', 'height': 854.0, 'width': 480.0, 'id': 7922}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:13 ==> GOD:365 home_or_office_furnishing_or_decor\lighting\ceiling_light
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:1 ==> GOD:378 home_or_office_furnishing_or_decor\seating\armchair
======furniture:22 ==> GOD:403 home_or_office_furnishing_or_decor\table_or_desk
======furniture:46 ==> GOD:399 home_or_office_furnishing_or_decor\storage\shelving
======furniture:42 ==> GOD:462 plant\potted_plant
======furniture:42 ==> GOD:462 plant\potted_plant
======furniture:42 ==> GOD

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '45498561D6E507E7A412266A2E2F74F445E18D16ACAC0DA76A3F3AFAC2B12247B81316AB37E0CD4512D07F3E7852D825714904BB73FB7D91BA9168CA4A7DEB8C.jpg', 'height': 640.0, 'width': 640.0, 'id': 28414}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:22 ==> GOD:403 home_or_office_furnishing_or_decor\table_or_desk
======furniture:42 ==> GOD:462 plant\po

{'file_name': '685040C4FE205F1D84D283E55376AF79327C918BB13985420346F2448EFF61B83D1D7C56A5A88EE22FCAF9D9519AE4F94A293B438CBAA369C4DDA762FFF369BE.jpg', 'height': 358.0, 'width': 358.0, 'id': 77685}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:49 ==> GOD:369 home_or_office_furnishing_or_decor\lighting\table_lamp
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '5630AECE11627CAA893A3F6CC84EAE0505524E29132D63E8FDCB00E40D9C50F43D5A91B3A597D6C0FFDE6B8C5C4D5AA02FFBB22A3D147DC37FAEA23AC644E339.jpg', 'height': 635.0, 'width': 450.0, 'id': 62650}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:9 ==> GOD:380 home_or_office_furnishing_or_decor\seating\bench
======furniture:22 ==> GOD:403 home

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '2DE61983F1B70ED0070CFB6E70E77002E714E3468443A0787B108A8C68EEC7BCC196CAA78143625125B3169755C8B854BA0156F86B683FCBB583BAEE5525EBC8.jpg', 'height': 300.0, 'width': 474.0, 'id': 75864}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
========COCO:9 ==> GOD:566:vehicle\water_vehicle\small_boat_or_yacht
========= Open Image:swimming pool ==> GOD:376:home_or_office_furnishing_or_decor\pool
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '4D2D0D403F64B7B931DBC21BC5736A9AE87DB1D51B222E26DC26454B545446C2D2286E717CFB718CB428219E321DD0764994C7A8E9D350CB9B5499CD14E69293.jpg', 'height': 287.0, 'width': 320.0, 'id': 81604}
img.file_name.type = <class 'str'>
img.height.type = <cl

...

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '284AB6C7258912D4AA649C34F3402F34CCA062AF6DD5408518BE522D3DA75C033C5D6D033AAB1115690DE2783612194EE3E72E6CA9F0C401C1FEFEA76D1F946E.jpg', 'height': 213.0, 'width': 320.0, 'id': 16053}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:22 ==> GOD:403 home_or_office_furnishing_or_decor\table_or_desk
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:14 ==> GOD:381 home_or_office_furnishing_or_decor\seating\chair
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:42 ==> GOD:462 plant\potted_plant
=

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '38E80C310FB4798461711CC25DE77B2573134CBF550E0105353B73AC0CBB77743031C5C1E089DEF9BC4AA2E24C6EE6C9BD497F554B854AE93C6473D2BBF9B139.jpg', 'height': 733.0, 'width': 550.0, 'id': 69920}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:42 ==> GOD:462 plant\potted_plant
======furniture:37 ==> GOD:398 home_or_office_furnishing_or_decor\storage\nightstand
======furniture:16 ==> GOD:331 home_or_office_furnishing_or_decor\comforter
====

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '03E712F5AC30035F44D38D30E253BB1171EF2CE6FE608953BBA10C925496F7116BDCC3094EDFE2EC5B9A3821ADD8405769BAB31D94FFDBDB65D733EA16B4CB34.jpg', 'height': 951.0, 'width': 736.0, 'id': 79953}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:36 ==> GOD:370 home_or_office_furnishing_or_decor\mirror
======furniture:42 ==> GOD:462 plant\potted_plant
======furniture:16 ==> GOD:331 home_or_office_furnishing_or_decor\comforter
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
========= Open Image:house ==> GOD:420:man_made_structure\building\residential_house_exterior_structure\house
furniture_GT
Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': 'E58F4A3C5A8D8B44508B48809A27185C24D2167221949A48FA00A7FC33F1A9ACCFB84DE737BD835DD266A4633632A69BC69F1A5CB50334AAF0FCD1FFA3843420.jpg', 'height'

Fashion_Detector
COCO_Mask_Detector
OpenImage_Detector
{'file_name': '596B72159A225F394B380230503D7C8C75370782970E464E3F519F0C0E6A8892B0E4F9B16FF122FE9674CB66ECF35A2DE21A8A60DFB4A1978F1758B9E7B46F8A.jpg', 'height': 187.0, 'width': 280.0, 'id': 22527}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:12 ==> GOD:125 electronics\home_appliances\fan
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:28 ==> GOD:361 home_or_office_furnishing_or_decor\frame_and_picture
======furniture:16 ==> GOD:331 home_or_office_furnishing_or_decor\comforter
======furniture:49 ==> GOD:369 home_or_office_furnishing_or_decor\lighting\table_lamp
======furniture:55 ==> GOD:145 electronics\home_entertainment\tv
======furniture:22 ==> GOD:403 home_or_office_furnishing_or_decor\table_or_desk
======furniture:7 ==> GOD:323 home_or_office_furnishing_or_decor\bed
======furnit

COCO_Mask_Detector
OpenImage_Detector
{'file_name': '5B6A867E8805A922C35A57C35321839BA9F9B176C3D41BCDA577F7CBA16988F028569B39FF4846FFFF91A2D8CC3E4EFB20D3773BF254503783EB7AA4C8EDCFBC.jpg', 'height': 2117.0, 'width': 600.0, 'id': 60008}
img.file_name.type = <class 'str'>
img.height.type = <class 'float'>
img.width.type = <class 'float'>
img.id.type = <class 'int'>
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
======furniture:39 ==> GOD:373 home_or_office_furnishing_or_decor\pillow_or_cushion\paper_decoration
========= Open Image:owl ==> GOD:25:animal\bird\owl
========= Open Image:butterfly ==> GOD:9:animal\arthropod\insect\butterfly
========= Open Image:butterfly ==> GOD:9:animal\arthropod\insect\butterfly
========= Open Image:penguin 

In [ ]:
print (data['images'][:10])
print ("Saving annotatinos to {}".format(outAnnoFile))
with open(outAnnoFile,'w') as fout:
    json.dump(data, fout)
print ("Done")